<a href="https://colab.research.google.com/github/dlsun/pods/blob/master/11-Hierarchical-Data/11.1%20The%20JSON%20Data%20Format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 11. Hierarchical Data

A lot of data in the real world is naturally hierarchical. As an example, consider a data set where each observation is a TV show. Many of the variables in this data set are what we would expect, such as the runtime of each show and what network it was broadcast on. But there are also variables that are unorthodox, such as the season and the cast. A TV show can have multiple seasons and multiple cast members, as illustated in the figure below.

![](https://github.com/dlsun/pods/blob/master/11-Hierarchical-Data/hierarchical_data.png?raw=1)

Can we represent all of this information in a single `DataFrame`? If each row represents a single show, then it is straightforward to have columns containing the runtimes, the premiere dates, and so on. But it is not obvious how to incorporate the season information into this `DataFrame` in a way that still makes this information accessible for analysis. There are multiple challenges:

- A show has multiple seasons. We will need multiple columns, one for each season.
- The number of seasons varies from show to show. For example, "Girls" was on for 6 seasons, while "The Golden Girls" was on for 7. We will need to have at least 7 columns in our `DataFrame` to be able to store all the information for "The Golden Girls", even though we only need 6 columns for "Girls" (and perhaps even fewer for other shows).
- Each season has multiple variables associated with it, such as the premiere date and the end date. We will need a separate column for each of these variables.

The resulting `DataFrame` might look something like this.

|name    | runtime |  premiered  | season1premiere | season1end | ... | season7premiere | season7end | 
|--------|---------|-------------|-----------------|------------|-----|-----------------|------------|
| Girls | 30       | 2012-04-15  | 2012-04-15      | 2012-06-17 | ... | `NaN`           | `NaN`      |
| The Golden Girls | 30 | 1985-09-14 | 1985-09-14 | 1986-05-10 | ... | 1991-09-21 | 1992-05-09      |
| ... | ... | ... | ... | ... | ... | ... | ... |

Furthermore, each season contains a different number of episodes. If we also want to store information about each episode, it is impractical to do so in a `DataFrame`.

The problem is that this data is naturally hierarchical. A TV show can have multiple cast members and multiple seasons; furthermore, each season can have multiple episodes. Hierarchical data requires a different storage format, which we explore in this chapter.

# Chapter 11.1 The JSON Data Format

The JavaScript Object Notation, or **JSON**, data format is a popular way to represent hierarchical data. Despite its name, its application extends far beyond JavaScript, the language for which it was originally designed.

Let's take a look at the first 1000 characters of a JSON file. (_Warning:_ Never try to print the entire contents of a JSON file in a Jupyter notebook; this will freeze the notebook if the file is large!)

In [0]:
# Fetch data from a URL
import requests
response = requests.get("https://dlsun.github.io/pods/data/tvshows.json")

print(response.text[:1000])

This syntax should seem familiar if you are a regular user of Python. Except for a few cosmetic differences, this is exactly the syntax of a Python dictionary! The `json` library in Python automatically translates a JSON string or file into a Python dict.

In [0]:
import json
data_shows = json.loads(response.text)

If the JSON is read in from a URL using the `requests` library, then the JSON object can also be accessed directly from the response.

In [0]:
# This code is equivalent to the above code.
data_shows = response.json()

Now let's investigate the JSON data that we just loaded, again being careful not to print out all of data. Let's start by looking at the top-level variables associated with each TV show.

In [0]:
show = data_shows[0] # data for the show "Girls"
show.keys()

We see variables like **runtime** and **premiered**, but also "variables" like **cast** and **seasons**, which contain multiple values.

In [0]:
show["runtime"]

In [0]:
show["cast"]

A "variable" (like **cast**) with multiple values is called a _repeated field_. A repeated field might itself contain a repeated field (e.g., each show has multiple seasons, and each season in turn has multiple episodes), creating a hierarchy of variables. Repeated fields are represented as lists or arrays in JSON.

Let's take a closer look at how each cast member is represented, by zooming in on the first cast member.

In [0]:
show["cast"][0]

It appears that each cast member is itself a dictionary with four keys: **person** (i.e., the actor), **character**, **self**, and **voice**. The first two attributes are themselves dictionaries containing further information about the actor and the character, while the last two attributes are booleans.

If we wanted to get the complete list of actors who appeared in these shows, excluding voice actors, we could traverse the levels using nested loops:

In [0]:
actors = []
for show in data_shows:
    for cast in show["cast"]:
        # exclude voice actors
        if not cast["voice"]:
            actors.append(cast["person"]["name"])
            
actors

However, it is often easier to work with hierarchical data by first flattening it to a `DataFrame`.

# Flattening Hierarchical Data

Although hierarchical data cannot be efficiently represented using a `DataFrame`, most questions do not require working with the full data. In these cases, it is helpful to first "flatten" the JSON data into a `DataFrame`.

For example, suppose we want to know the average runtime of shows. To answer this question, it suffices to work with a `DataFrame` with one row per show. We can use the `json_normalize` function in `pandas` to flatten the data into a `DataFrame` of this form.

In [0]:
import pandas as pd
from pandas.io.json import json_normalize

df_shows = json_normalize(data_shows)
df_shows

In [0]:
df_shows["runtime"].mean()

Let us take a closer look at the columns of this `DataFrame`.

In [0]:
df_shows.keys()

Notice that variables that were themselves dictionaries, such as **schedule** and **network**, have been expanded into multiple columns, with names like **schedule.time**, **schedule.days**, etc.

Repeated fields, like **genres**, **cast**, and **seasons**, are also columns in this `DataFrame`. These columns just contain a dump of the raw JSON. The information in these columns is not readily accessible.

In [0]:
df_shows["seasons"]

What if we wanted to identify the show with the most episodes? It is difficult to calculate this from the `DataFrame` above, since the episodes are buried within the **seasons** column. It would be preferable to have a `DataFrame` where each row represents a season of a show.

The `json_normalize()` function also accepts an additional argument specifying the variable that we want to be the rows of the `DataFrame`. So if we wanted a `DataFrame` where each row represents a season, we would pass in the name of that variable in the JSON data (i.e., **seasons**) to `json_normalize()`.

In [0]:
df_seasons = json_normalize(data_shows, "seasons")
df_seasons

There is just one problem. We now have a `DataFrame` of seasons, without any indication of which TV show they came from. This is because when we used `json_normalize()`, it automatically discarded variables from all levels above the one that we flattened to. (Since we flattened to the "season" level, we lost all variables associated with the "show".) If there are any variables from the higher levels that we want to keep, then they have to be specified explicitly in the `meta=` argument. Since we want the name of the TV show, which is stored in the "name" key of each show, we specify `meta="name"`.

(We also specify a prefix in the `meta_prefix=` argument to avoid column name clashes, since the `DataFrame` already has a column called **name**. This is not necessary if the column names do not clash.)

In [0]:
df_seasons = json_normalize(data_shows, "seasons", meta="name", meta_prefix="show.")
df_seasons

From here, it is straightforward to calculate the total number of episodes for each show. First, we determine the number of episodes in each season by calculating the length of each episodes list, storing the result in a new column called **num_episodes**. Then, we calculate the sum of **num_episodes** for each **show.name**.

In [0]:
df_seasons["num_episodes"] = df_seasons["episodes"].apply(len)
df_seasons.groupby("show.name")["num_episodes"].sum()

Alternatively, we could have answered this question by flattening the JSON data to the episode level. Since "episodes" are nested underneath "seasons", we have to specify the path to the "episodes" variable in the JSON data. 

In [0]:
df_episodes = json_normalize(data_shows, ["seasons", "episodes"], meta="name", meta_prefix="show.")
df_episodes

Since the observational unit is already an episode, we simply count the number of times each show appears in this `DataFrame`.

In [0]:
df_episodes["show.name"].value_counts()

# RESTful Web Services

One way that organizations expose their data to the public is through RESTful web services. In a typical RESTful service, the user specifies the desired data in an HTTP request, and the server responds with the requested data. JSON is a common format for the response data.

For example, the JSON data that we have been analyzing in this lesson was retrieved from the [TVMaze API](http://www.tvmaze.com/api). Most APIs come with accompanying documentation explaining how to construct HTTP requests to fetch data. For example, to query the TVMaze API for TV shows related to the term "office", we would issue a request to the following URL: http://api.tvmaze.com/search/shows?q=office. Try visiting this URL in a browser; you should see a long string of JSON instead of a rendered webpage!

We can import this JSON into our Python session using the `requests` library, as above.

In [0]:
import requests
response = requests.get("http://api.tvmaze.com/search/shows?q=office")
data_office = response.json()

# Print the first 1000 characters.
str(data_office)[:1000]

This JSON can then be processed using the techniques discussed above.

Although many RESTful APIs work similarly, there is no universal standard, so you will usually have to carefully read the documentation for the API that you want to use.

# Ethical Enlightenment: Staggering Requests

Suppose we want information about the individual episodes of each show we found above.

In [0]:
df_office = json_normalize(data_office)
df_office

From [the documentation](http://www.tvmaze.com/api#show-episode-list), we see that the episodes can be retrieved using the ID in the **show.id** column, by constructing a URL of the form http://api.tvmaze.com/shows/[ID]/episodes.

It is straightforward enough to write a loop that replaces [ID] in this URL by the actual ID of each show. However, a script can easily issue hundreds, even thousands, of queries per second, and we want to avoid spamming the server. In fact, most RESTful services have [rate limiting policies](http://www.tvmaze.com/api#rate-limiting), which means that they automatically block users if they receive too many requests from that user within a window of time. Many RESTful services also require that API keys be supplied with every request, allowing the website to block the API keys of abusers.

Out of respect for the host, who is often providing this service for free, we stagger our requests by inserting a time delay in our code. This can be done using `time.sleep()`, which will suspend execution of the script for the given number of seconds. We will add a half second delay (so that we make no more than 2 queries per second) between requests.

In [0]:
import time

episodes = []
for show_id in df_office["show.id"]:
    
    # get the episodes for the show from the REST API
    response = requests.get("http://api.tvmaze.com/shows/%d/episodes" % show_id)
    episodes.extend(response.json())
    
    # add a 0.5 second delay between each query
    time.sleep(0.5)
    
# Now we have a list of episodes in JSON format.
# We can convert this to a DataFrame of episodes using json_normalize.
json_normalize(episodes)

# Exercises

Exercises 1-2 ask you to use the [TVMaze API](http://www.tvmaze.com/api) from the lesson.

1\. What was the longest show that aired in the U.S. on February 3, 2019?

(_Hint:_ Use the ["Schedule" endpoint](http://www.tvmaze.com/api#schedule).)

2\. Among all shows that aired in the U.S. on Feburary 3, 2019, only one person appeared on more than one show. (You may find three people. Two of them actually appeared in two different roles on the same show.) Who was it?

(_Hint_: Use the **show.id** from the data above, and use the ["Shows" endpoint](http://www.tvmaze.com/api#show-cast) to get the cast of each show. Don't forget to stagger your requests, or you will be blocked by the website!)

To answer Exercises 3-5, you will need to issue HTTP requests to the Open States API, which contains information about state legislatures. Unlike the TVMaze API, this one requires an API key with every request. You can [register for an API key here](https://openstates.org/api/register/). Once you have an API key, enter your API key below. If your API key works, then the code below should produce a `DataFrame` of the legislators in the California State Assembly (the lower chamber).

In [0]:
# This is just a sample request to test that your API key is working.
apikey = "6ca8c1d0-d4a2-4c48-9211-ab28b6719d65"
resp = requests.get(
    "https://openstates.org/api/v1/legislators/?state=ca&chamber=lower&apikey=%s" % apikey
)
json_normalize(resp.json())

To answer the questions below, you will need to issue your own HTTP requests to the API. To understand how to construct URLs, you will need to refer to the [documentation for this API](http://docs.openstates.org/en/latest/api/).

3\. Legislators typically have offices in both the Capitol building and in their districts. Among the active legislators in the California Assembly (lower chamber), which legislators have the most offices (and how many do they have)?

4\. Get all of the _constitutional amendments_ in the California State Senate (upper house) from the current legislative session. How many amendments have there been?

(_Hint:_ "Constitutional amendment" is a type of bill.)

5\. Look up the votes on the constitutional amendments you found in Exercise 5. Calculate the number of "yes" and "no" votes for each legislator on these amendments. Which legislator had the most total votes on constitutional amendments in the current session? Which legislator had the most total negative votes?